# Indépendance

On va ici faire plus de simulation que dans la partie dénombrement, on utilisera ainsi beaucoup la librairie random de python, et aussi peut-être, numpy.random .

In [ ]:
import numpy as np
from random import randint

#Imaginons qu'on lance 2 dé :
(randint(1, 6), randint(1, 6))

In [ ]:
#Si l'on fait maintenant 100 000 lancés :
jets_comprehension = [(randint(1,6), randint(1,6)) for _ in range(100000)]

#On vous montre ici une façon très mauvaise de le faire !!

#Il aurait en effet était préférable d'utiliser numpy :

#ATTENTION, le randint de numpy est exclusif ([a, b[), alors que celui natif en python est inclusif ([a, b])
#on utilise donc random.randint(1, 6) mais numpy.random.randint(1, 7) !

jets_numpy = np.random.randint(1, 7, (100000, 2))

In [ ]:
%timeit [(randint(1,6), randint(1,6)) for _ in range(100000)]
%timeit np.random.randint(1, 7, (100000, 2))
#On voit ici que la version avec numpy est environ 50 fois plus rapide

print(f'\nLes 5 premiers jets de notre tableau : \n{jets_comprehension[:5]}')
print(f'\nLes 5 premiers jets du tableau retourné par numpy : \n{jets_numpy[:5]}\n')

In [ ]:
#on peut alors faire un peu ce que l'on veut avec notre echantillonage:

#la formule ici a l'air un peu barbare, mais en vrai rien de bien dur à comprendre !

print("Avec np.sum et le parcours avec numpy")
%timeit np.sum(np.all(jets_numpy[:] == (1, 1), axis = -1)) / len(jets_numpy)

#normalement, si on fait np.array([1,2]) == np.array([1,2]), on aura [True, True]
#si maintenant on fait   np.array([1,2]) == np.array([2,2]), on aura [False, True]

#Ainsi, nou on cherche à voir si toutes les cases correspondent, ainsi, on fait un np.all
#ce qui nous assure qu'on a [True, True].

#Ainsi, jets_numpy[:] nous renvoie tous les sous-tableaus de 2 cases,
#et on regarde s'ils sont égaux à (1, 1) en utilisant np.all.


print("\nAvec sum et le parcours avec numpy")
%timeit sum(np.all(jets_numpy[:] == (1, 1), axis = -1)) / len(jets_numpy)


print("\nAvec le parcours en python natif")
%timeit len([0 for jet in jets_numpy if jet[0] == 1 and jet[1] == 1]) / len(jets_numpy)

proba_empirique = np.sum(np.all(jets_numpy[:] == (1, 1), axis = -1)) / len(jets_numpy)



#on peut simplement faire :
print(f'\n\nproba empirique : {proba_empirique}\nvrai proba : {1/36}')

Quand on voit la différence de temps, il faut vraiment penser à utiliser np.sum sur les arrays numpy ! La majorité des fonctions de numpy sont en fait directement codées en C / C++ et sont précompilées. Ainsi, sur des tableaux compatibles (les arrays numpy, aussi appelés ndarray), il est vraiment très préférable d'appeler des fonctions optimisées, prévues à cet effet, plutôt que les fonctions natives de python.

In [ ]:
#ou alors compter toutes les possibilités :
from collections import Counter
d = Counter([(x, y) for x, y in jets_numpy]) #permet de compter le nombre de répétitions de chaque objet différents
print(f'Nombre de (1, 1) : {d[(1,1)]}\nNombre de (2, 2) : {d[(2,2)]}\nNombre de (4, 6) : {d[(4,6)]}')

In [ ]:
#Soit nos jets : (X, Y), si on veut approximer P(Y = 1 | X = 1) :

nb_jets_total = sum([d[(1, i)] for i in range(1, 7)]) #nombre total de jet où X = 1
proba_empirique = d[(1,1)] / nb_jets_total

print(f'proba empirique : {proba_empirique}\nvrai proba : {1/6}')

In [ ]:
#si maintenant on regarde le nombre de fois où on a eu un 1 :

p_x = sum([d[(1, i)] for i in range(1, 7)]) / len(jets_numpy) #P(X = 1)
p_y = sum([d[(i, 1)] for i in range(1, 7)]) / len(jets_numpy) #P(Y = 1)

print(f'proba empirique : {p_x + p_y}')

#si vous êtes attentifs un truc devrait vous déranger

In [ ]:
# On voit que la proba empirique approche 1/3

# or la proba de n'avoir aucun 1 étant normalement de 5/6 * 5/6 = 25/36, on devrait avoir une proba empirique de 11/36 soit environ 1/3, mais pas tout à fait...
# 1/3 = 0.3333 alors que 11/36 = 0,305555.
# ici, on approche clairement plus de 1/3 du coup... Et c'est là que ça doit tout de suite vous alertez, on a trop de valeurs !
# tout à l'heure on a prit tous les (1, y) et (x, 1) et on a donc compté les (1, 1) 2 fois.
# Comme les (1,1) représentent 1/36 de l'échantillon total, si on les prend en compte 2 fois, on a alors (11/36 + 1/36 soit 1/3 de notre échantillon), tout est logique.
print(f'proba empirique : {p_x + p_y - d[(1,1)] / len(jets_numpy)}')

On trouve donc une proba beaucoup plus proche de ce que l'on attendait, donc approximez, mais pas trop en cours de stats (gardez ça pour la vie de tous les jours).
Les 2 évènements au-dessus étant clairement indépendanst, enfin ça dépend de la génération aléatoire qui est probablement seulement pseudo-aléatoire, mais bon... C'est indépendant, ou presque, ...

Truc sympa quand vous voulez approximer une proba ou un 
pourcentage dans la vie de tous les jours, l'erreur du calcul entre
la multiplication et l'addition est souvent assez faible :

Imaginons :
* pour $1.05 * 1.05 = 1.1 + 0.05 * 0.05$
* pour $5/6 * 5/6 = 2/3 + 1/6 * 1/6$

Ainsi au dessus de 1, c'est une addition, en dessous, une soustraction, et on rajoute l'erreur si les deux sont au-dessus ou en-dessous de 1 ou on la soustrait si l'un est au-dessus de 1 et l'autre en-dessous. Erreur qui est alors la multiplication de la distance à 1 des 2 chiffres.

Par exemple : 
* $1.1 \times 1.2 \ \ = \  1.3 + 0.02\ = 1.32$
* $1.1 \times 0.9 \ \ = \ 1 - 0.01 \ \ \ \ = 0.99$
* $0.6 \times 0.8 \ \ =\  0.4 + 0.08 \ = 0.48$
* $1.25 \times 0.8 = 1.05 - 0.05 = 1$

Pour aussi partir un peu plus loin quand vous faisiez des dérivés en terminal (si jamais), vous auriez sûrement aimé connaitre : 
https://en.wikipedia.org/wiki/Automatic_differentiation#Automatic_differentiation_using_dual_numbers

Ca paraît compliqué, car c'est la définition formelle mais en vrai, c'est hyper simple, soit votre variable $x + \epsilon$ en argument, vous développez tout, et vous dîtes par définition que $\epsilon \rightarrow 0$ mais n'est pas égal à 0. Cependant, $\epsilon^2$ étant extrêmement plus proche de 0 que $\epsilon$, il n'est pas tant incorrect de dire que $\epsilon^2 << \epsilon$ est on peut donc l'ignorer.

Par exemple : pour $f(x) = x^2$, on a alors $f(x + \epsilon) = (x + \epsilon)^2 = x^2 + 2x\epsilon + \epsilon^2 \simeq x^2 + 2x\epsilon$ soit la fonction normal ($x^2$) plus sa dérivée ($2x$) multiplié par $\epsilon$.

Cette équation est très liée à $f'(x) = lim_{h \rightarrow 0}\ \ \frac{f(x + h)}{h}$, la définition de la dérivée.

Enfin bref, les erreurs très petites peuvent souvent être ignorées quand elles sont au carré, très pratique pour approximer !

# Dépendance

Si on a maintenant deux variables aléatoires, mais que l'une dépend de l'autre :

In [ ]:
# Vous pouvez utiliser ctrl + entrée pour exécuter une cellule plusieurs fois d'affilé
# ou shift + entrée pour exécuter la cellule courante, puis passer à la suivante.
#soit par exemple, un truc assez classique :
x = randint(1, 6)
y = randint(1, x)
(x, y)

In [ ]:
from collections import Counter
tirages = [(x, randint(1, x)) for x in [randint(1, 6) for _ in range(100000)]]

d = Counter(tirages)

d = {k : v / len(tirages) for k, v in d.items()}
d[(1,1)]

In [ ]:
#On peut alors donner la répartition empirique de la loi jointe
p = np.zeros((6,6))
for (i, j), v in d.items():
    p[i - 1, j - 1] = v
print(p)

Si on veut maintenant la loi de X = i, sachant que Y = j :

On applique la formule, directement, on a alors :
    $P(X = i | Y = j) = \frac{P(X = i, Y = j)}{P(Y = j)}$

In [ ]:
#On commence par calculer la loi de Y
p_y = sum(p)
print(p_y)

In [ ]:
#Ainsi, on peut avoir la loi de X, sachant Y :
p_x = p / p_y
print(p_x)
# On voit alors, même si on le savait depuis le début que X et Y ne sont pas du tout indépendant...

Un autre problème, assez connu des statistiques, est le suivant :

Vous êtes testé positif à une maladie très rare, quelle est à posteriori la probabilité que vous ayez vraiment cette maladie ?

https://www.youtube.com/watch?v=lG4VkPoG3ko

La réponse est alors, tout dépend de la rareté de la maladie et de l'accuracy du test.

Si nous simulons un échantillon de 10 000 personnes :


In [ ]:
from random import random

#soit p l'accuracy du test (en anglais, accuracy et precision sont deux données bien distinctes.) :
# https://en.wikipedia.org/wiki/Precision_and_recall
p = 0.99

#soit apriori, la probabilité d'avoir la maladie :
apriori = 0.01

#On a alors le groupe positif :
positif = 0
negatif = 0
for _ in range(10000):
    if random() < apriori:
        positif += 1
    else:
        negatif += 1
print(f'Positifs : {positif}\nNégatifs : {negatif}')



In [ ]:

vrai_positif = 0
faux_positif = 0
vrai_negatif = 0
faux_negatif = 0
for _ in range(positif):
    if random() <= p:
        vrai_positif += 1
    else:
        faux_negatif += 1

for _ in range(negatif):
    if random() <= p:
        vrai_negatif += 1
    else:
        faux_positif += 1


print(f'Vrai positifs : {vrai_positif}\nFaux positifs : {faux_positif}\nPositifs au tests : {vrai_positif + faux_positif}\n\nVrai négatifs : {vrai_negatif}\nFaux négatif : {faux_negatif}\nNégatifs au tests : {vrai_negatif + faux_negatif}')

Il faut voir ces stats comme:

Il est très peu probable que vous ayez cette maladie, ainsi si le test vous donne négatif, alors vous êtes presque sûr de d'être vraiment négatif (ici, notre chance d'être Faux Négatif est égale à $\frac{Faux\ Négatif}{Négatif\ Au\ Test}$ (2/9809 avec mes données)), alors que si vous êtes testé positif, il est autant probable que le teste dise n'importe quoi ou que vous ayez vraiment la maladie (99/191 dans mes données). 

Cela est dû aux valeurs opposés de l'accuracy et de la rareté de la maladie, si on change le ratio, alors on changera l'information que nous apporte ce test. On voit assez bien que si la maladie est plus rare, il y aura en fait plus de chance que le test rate, plutôt que vous ayez vriament la maladie. Ainsi, pour des valeurs un peu plus extrême pour la probabilité à priori d'avoir la maladie, le test n'apporte que très peu d'information.

Si on va un peu plus loin, on parle ici d'un test qu'on fait avec l'apriori de base, c'est à dire que vous avez autant de chance d'avoir cette maladie que quelqu'un d'autre, mais, dans un contexte plus réel, si vous demandez à vous faire dépister, ou que votre médecin décide de vous dépister, vous n'avez généralement pas vraiment l'apriori de base. En effet, les symptômes, les gênes, et tous les autres facteurs de risques peuvent venir appuyer le test, (en locurence c'est plutôt le test qui vient appuyer ces données).

Le résultat du test viendra alors update vos chances de vraiment avoir cette maladie. Il ne faut pas voir le résultat comme une réponse, mais comme une indication (plus ou moins fiable).

In [ ]:
print(f"Chance d'être positif sachant que le test est positif : {vrai_positif/(vrai_positif + faux_positif)}  (C'est ça que l'on appelle precision en anglais)")
print(f"Chance d'être négatif sachant que le test est négatif : {vrai_negatif/(vrai_negatif + faux_negatif)}")
print(f"Chance d'être testé positif sachant que vous êtes positif : {vrai_positif/(vrai_positif + faux_negatif)} (C'est ça que l'on appelle recall en anglais)")